In [2]:
!pip install tqdm

In [5]:
!pip install transformers accelerate bitsandbytes>0.37.0 datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
import time
import numpy as np
import torch

# Load the tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load and configure models with BitsAndBytesConfig
quantization_config_8bit = BitsAndBytesConfig(load_in_8bit=True)
quantization_config_4bit = BitsAndBytesConfig(load_in_4bit=True)

# Load models with 8-bit and 4-bit quantization
model_fp32 = AutoModelForCausalLM.from_pretrained(model_name)
model_8bit = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config_8bit)
model_4bit = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config_4bit)

# Print memory footprints using `get_memory_footprint`
print(f"FP32 Model Memory Usage: {model_fp32.get_memory_footprint() / (1024 ** 3):.2f} GB")
print(f"8-bit Model Memory Usage: {model_8bit.get_memory_footprint() / (1024 ** 3):.2f} GB")
print(f"4-bit Model Memory Usage: {model_4bit.get_memory_footprint() / (1024 ** 3):.2f} GB")

# Measure inference latency
input_text = "The history of quantum mechanics begins with"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Latency measurement for FP32
start_time = time.time()
with torch.no_grad():
    model_fp32(input_ids)
latency_fp32 = time.time() - start_time
print(f"FP32 Model Latency: {latency_fp32:.2f} seconds")

# Latency measurement for 8-bit model
start_time = time.time()
with torch.no_grad():
    model_8bit(input_ids)
latency_8bit = time.time() - start_time
print(f"8-bit Model Latency: {latency_8bit:.2f} seconds")

# Latency measurement for 4-bit model
start_time = time.time()
with torch.no_grad():
    model_4bit(input_ids)
latency_4bit = time.time() - start_time
print(f"4-bit Model Latency: {latency_4bit:.2f} seconds")

# Load a subset of the Wikipedia dataset
dataset = load_dataset("wikipedia", "20220301.en", split="train[:3000]")

# Helper function to compute perplexity
def calculate_perplexity(model, tokenizer, dataset):
    texts = [entry["text"] for entry in dataset]
    encodings = tokenizer("\n\n".join(texts), return_tensors="pt", truncation=True, max_length=512)
    stride = 512
    nlls = []
    for i in range(0, encodings.input_ids.size(1), stride):
        input_ids = encodings.input_ids[:, i:i + stride].to(model.device)
        target_ids = input_ids.clone()
        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            nll = outputs.loss.item() * input_ids.size(1)
            nlls.append(nll)
    perplexity = np.exp(np.sum(nlls) / len(encodings.input_ids[0]))
    return perplexity

# Compute perplexity for FP32, 8-bit, and 4-bit models
print("Calculating perplexity...")
perplexity_fp32 = calculate_perplexity(model_fp32, tokenizer, dataset)
perplexity_8bit = calculate_perplexity(model_8bit, tokenizer, dataset)
perplexity_4bit = calculate_perplexity(model_4bit, tokenizer, dataset)

print(f"FP32 Model Perplexity: {perplexity_fp32:.2f}")
print(f"8-bit Model Perplexity: {perplexity_8bit:.2f}")
print(f"4-bit Model Perplexity: {perplexity_4bit:.2f}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


FP32 Model Memory Usage: 0.48 GB
8-bit Model Memory Usage: 0.16 GB
4-bit Model Memory Usage: 0.12 GB
FP32 Model Latency: 2.10 seconds
8-bit Model Latency: 3.74 seconds
4-bit Model Latency: 0.10 seconds


/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

wikipedia.py:   0%|          | 0.00/36.7k [00:00<?, ?B/s]

...

train-00038-of-00041.parquet:   0%|          | 0.00/203M [00:00<?, ?B/s]

train-00039-of-00041.parquet:   0%|          | 0.00/192M [00:00<?, ?B/s]

train-00040-of-00041.parquet:   0%|          | 0.00/185M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6458670 [00:00<?, ? examples/s]

Calculating perplexity...
FP32 Model Perplexity: 24.92
8-bit Model Perplexity: 25.05
4-bit Model Perplexity: 27.51
